# Example Interface Simulation Setup

This is a minimal working example showing how to set up a molecular dynamics (MD) simulation using a DeepMD-trained neural network potential to model oxygen diffusion in a Si/SiO₂ interface system.

The simulation involves a crystalline silicon substrate bonded to an amorphous SiO₂ layer. Initially, silicon atoms are frozen to maintain the crystalline structure, while the SiO₂ region is melted, quenched, and equilibrated. After controlled cooling, the entire system is relaxed, and a final energy minimization is performed.

## Key Features

- DeepMD potential used for interatomic interactions
- Selective freezing of silicon atoms during heating and melting
- NPT ensemble with directional pressure control (`z`-axis)
- Gradual heating, equilibration, cooling, and release of constraints
- Final structure relaxed via energy minimization

## Files Generated

For each run (e.g., `run_01/`, `run_02/`, ...):

- `in.lammps`: LAMMPS input file generated with custom temperature and random seed
- `submit.sh`: SLURM job script for running on HPC clusters
- Multiple `.lammpstrj` trajectory files (e.g., `dump_heat`, `dump_eq`, `dump_cool`, ...) saved during key stages

## Parameters

- `tmax`: Peak simulation temperature during melting (default: `2000 K`)
- `tunfreeze`: Temperature where constraints are released (`1500 K`)
- `nsim`: Number of independent simulations generated (with different seeds)

## How to Use

1. Place your initial interface structure file (e.g., `Si_110_SiO2_alpha_586.data`) in the expected location.
2. Update the `structure_file` path in the Python script if needed.
3. Run the script to generate folders like `run_01/`, ..., `run_10/`.
4. Submit each simulation to your SLURM-managed HPC using the provided `submit.sh`.


In [ ]:
import numpy as np
from pathlib import Path

# Simulation parameters
tmax = 2000            # Maximum temperature during melting phase [K]
tunfreeze = 1500       # Temperature at which frozen atoms are released [K]
nsim = 10              # Number of independent MD simulations (different seeds)

# Paths
output_root = Path("./lammps_md_110")  # Output folder for all runs
structure_file = "../../structures/Si_110_SiO2_alpha_586.data"  # Input structure

# Template for LAMMPS input script
lammps_template = """# LAMMPS input for Si/SiO2 interface melting/quenching

units           metal
atom_style      atomic
atom_modify     map yes
newton          on

boundary        p p p

read_data       {structure_file}

mass            1 15.999
mass            2 28.0855

# Replace this with the path to your model
pair_style      deepmd /projappl/jeakola/ekvikne/deepmd_models/25-05-13_model_oxygen_diffusion_main.pb
pair_coeff      * * O Si

neighbor        2.0 bin
neigh_modify    delay 0 every 1 check yes

thermo          1000
thermo_style    custom step temp etotal press vol

# Define frozen region (crystalline Si slab)
region freeze_region_lower block INF INF INF INF INF 20.0 units box
region freeze_region_upper block INF INF INF INF 48.0 INF units box

group freeze_lower region freeze_region_lower
group freeze_upper region freeze_region_upper
group frozen_atoms union freeze_lower freeze_upper

fix freeze frozen_atoms setforce 0.0 0.0 0.0
group mobile_atoms subtract all frozen_atoms

# Initial velocity creation
timestep        0.001
velocity        mobile_atoms create 300 {seed} dist gaussian

# Heat up mobile atoms to tmax
fix             npt_heat mobile_atoms npt temp 300 {tmax} 0.2 z 0.0 0.0 1.0
dump            dump_heat all custom 1000 dump_heat.lammpstrj id type x y z
run             40000
unfix           npt_heat
undump          dump_heat

# Hold at high temperature
fix             npt_eq mobile_atoms npt temp {tmax} {tmax} 0.2 z 0.0 0.0 1.0
dump            dump_eq all custom 1000 dump_eq.lammpstrj id type x y z
run             60000
unfix           npt_eq
undump          dump_eq

# Snapshot after high-T equilibration
dump            dump_post_high_T_eq all custom 1 post_high_T_eq.lammpstrj id type xu yu zu vx vy vz
run             0
undump          dump_post_high_T_eq

# Cool system to tunfreeze
fix             npt_cool mobile_atoms npt temp {tmax} {tunfreeze} 0.2 z 0.0 0.0 1.0
dump            dump_cool all custom 1000 dump_cool.lammpstrj id type x y z
run             40000
unfix           npt_cool
undump          dump_cool

# Snapshot just before unfreezing atoms
dump            dump_pre_unfreeze all custom 1 pre_unfreeze.lammpstrj id type xu yu zu vx vy vz
run             0
undump          dump_pre_unfreeze

# Release constraints (unfix frozen atoms)
unfix           freeze
fix             remove_drift all momentum 100 linear 1 1 1

# Short equilibration with all atoms unfrozen
fix             npt_eq_pos_unfreeze_all all npt temp {tunfreeze} {tunfreeze} 0.2 z 0.0 0.0 1.0
dump            dump_eq_pos_unfreeze_all all custom 500 dump_eq_pos_unfreeze_all.lammpstrj id type x y z
run             30000
unfix           npt_eq_pos_unfreeze_all

# Final cooling to 300K
fix             npt_cool_all all npt temp {tunfreeze} 300 0.2 z 0.0 0.0 1.0
dump            dump_cool_all all custom 500 dump_cool_all.lammpstrj id type x y z
run             30000
unfix           npt_cool_all

# Final room temperature equilibration
fix             npt_final_eq all npt temp 300 300 0.2 z 0.0 0.0 1.0
dump            dump_final_eq all custom 500 dump_final_eq.lammpstrj id type x y z
run             20000
unfix           npt_final_eq

unfix remove_drift

# Final energy minimization
min_style cg
minimize 1.0e-7 1.0e-9 1000 10000

write_data snapshot_minimized.data
"""

# Template for SLURM job submission script, replace the path following "source" with path to your env
submit_template = """#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=YOUR_ACCOUNT_NAME
#SBATCH --partition=small
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --time=2-1
#SBATCH --cpus-per-task=16
#SBATCH --mem=4GB
#SBATCH --output=output_%j.log
#SBATCH --error=error_%j.log

module load tensorflow
source /projappl/jeakola/sondredahl/DeePMD/bin/activate

srun lmp -sf omp -in in.lammps
"""

# === Generate multiple simulation folders === #
for i in range(1, nsim + 1):
    seed = i
    folder_name = f"run_{i:02d}"
    folder_path = output_root / folder_name
    folder_path.mkdir(parents=True, exist_ok=True)

    # Generate in.lammps file with seeded randomness
    in_script = lammps_template.format(
        structure_file=structure_file,
        tmax=tmax,
        tunfreeze=tunfreeze,
        seed=seed
    )
    with open(folder_path / "in.lammps", "w") as f:
        f.write(in_script)

    # Generate SLURM submit script
    job_name = folder_name
    submit_script = submit_template.format(job_name=job_name)
    with open(folder_path / "submit.sh", "w") as f:
        f.write(submit_script)

print("All 10 simulation folders created.")
